# 4. Doc2Vec

In [1]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
import nltk
from nltk.tokenize import word_tokenize

In [2]:
movie_df = pd.read_csv('/Users/meg/Thesis/movie_df_final.csv')
print(len(movie_df))

33520


### Tokenize

In [3]:
#tokenize with removing punctuation 
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
tagged_data = [TaggedDocument(words=tokenizer.tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(movie_df['Plot'])]

### PV-DM Hierarchical Softmax

In [4]:
modelpvdm = gensim.models.doc2vec.Doc2Vec(vector_size=200, window = 15, dm=1, hs=1, min_count=5, epochs=30)

In [5]:
print(modelpvdm.alpha)
print(modelpvdm.min_alpha)

0.025
0.0001


In [6]:
%%time

modelpvdm.build_vocab(tagged_data)
modelpvdm.train(tagged_data, total_examples = modelpvdm.corpus_count, epochs=modelpvdm.epochs)

vector_size=200
docvecs = np.zeros((modelpvdm.corpus_count, vector_size))
for i in range(modelpvdm.corpus_count):
    x = np.linalg.norm(modelpvdm.dv[i])
    docvecs[i] = modelpvdm.dv[i]/x
    
docvecsdf_modelpvdm = pd.DataFrame(docvecs)

CPU times: user 40min 52s, sys: 7.45 s, total: 40min 59s
Wall time: 13min 55s


In [7]:
docvecsdf_modelpvdm.to_csv('doc2vec_embeddings.csv')

In [8]:
print("Top 10 most similar to 'princess'")
modelpvdm.wv.most_similar('princess', topn=10) 

Top 10 most similar to 'princess'


[('prince', 0.6312201023101807),
 ('king', 0.5790107250213623),
 ('queen', 0.4999341666698456),
 ('duke', 0.41433629393577576),
 ('kingdom', 0.41289785504341125),
 ('crown', 0.3804285228252411),
 ('serpent', 0.36342090368270874),
 ('palace', 0.3627432584762573),
 ('slipper', 0.3567153215408325),
 ('goddess', 0.3470448851585388)]